In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import *
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
ratings_columns = ['userid', 'movieid', 'rating']
ratings = pd.read_csv(r"C:\Users\acer\Downloads\ml-100k\ml-100k\u.data", sep = '\t', names = ratings_columns, usecols = range(3))

In [3]:
#movie inputs (Taking 2 columns)
movie_columns = ['movieid', 'title']
movies = pd.read_csv(r"C:\Users\acer\Downloads\ml-100k\ml-100k\u.item", sep = '|', engine = 'python', names = movie_columns, usecols= range(2))

In [4]:
ratings.head()

,userid,movieid,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [5]:
movies.head()

,movieid,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [8]:
#merging both dataframes
dataset = pd.merge(ratings, movies)

In [9]:
dataset.head()

,userid,movieid,rating,title
0,196,242,3,Kolya (1996)
1,63,242,3,Kolya (1996)
2,226,242,5,Kolya (1996)
3,154,242,3,Kolya (1996)
4,306,242,5,Kolya (1996)


In [12]:
#Pivoting table to cerate a user-item-rating matrix
user_movies = dataset.pivot_table(index = 'userid', columns= 'title', values = 'rating')

In [13]:
user_movies.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
userid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN


In [27]:
# Taking the user item rating matrix and filling nans with 0s.
svd_movies = user_movies.fillna(0)
svd_movies.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
userid,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,2.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0


In [28]:
#Dividing each rating a user gave by the mean of each user's rating and creating a new user item rating matrix
user_means = np.array(svd_movies.mean(axis = 1)).reshape(-1, 1)
svd_movies = svd_movies.div(svd_movies.mean(axis = 1), axis = 0)
svd_movies_matrix = svd_movies.values

In [29]:
#getting the U, S and Vt values
U, sigma, Vt = svds(svd_movies_matrix, k = 10)

In [30]:
print('Left singular matrix: \n',U)

Left singular matrix: 
 [[ 0.00964187  0.01398517 -0.00166486 ...  0.01946524  0.0256143
   0.01347106]
 [ 0.00658166 -0.00577453 -0.00594398 ... -0.02381881  0.00400559
   0.04273718]
 [-0.03732838  0.05836725  0.02127472 ...  0.01297244 -0.03184454
   0.03761754]
 ...
 [ 0.03965977  0.03960672  0.04544805 ... -0.07473099  0.04091936
   0.04359049]
 [-0.02463194  0.0175321  -0.01070806 ...  0.03721523  0.01182754
   0.02333003]
 [-0.00919452 -0.00924768 -0.01627082 ...  0.01856391  0.03132799
   0.01238725]]


In [31]:
print('Right singular matrix: \n',Vt)

Right singular matrix: 
 [[-3.06392559e-04 -7.57265439e-04  1.46859489e-02 ... -1.62641643e-03
  -1.43177860e-03 -3.48926952e-07]
 [ 6.65716435e-04  2.70119817e-04  1.30707666e-02 ...  1.09414805e-03
   9.52235145e-04  1.17435346e-05]
 [-1.94839879e-03  8.94244342e-03 -3.74068104e-02 ... -2.76515250e-03
   7.56868519e-04 -2.43238037e-05]
 ...
 [ 5.45842099e-04 -1.25496431e-03 -1.71096904e-02 ... -1.65421804e-03
   1.76392472e-03  2.10973004e-05]
 [ 6.63718233e-04  1.46941219e-03  1.84169377e-02 ...  9.83215290e-04
  -1.59642483e-03  1.26671280e-05]
 [ 3.94811894e-04  6.13999926e-04  1.06738442e-02 ...  3.93993135e-04
   1.74772628e-03  8.61308218e-06]]


In [32]:
# Construct diagonal array in SVD
sigma = np.diag(sigma)
print('Diagonal matrix: \n',sigma)

Diagonal matrix: 
 [[ 745.00611247    0.            0.            0.            0.
     0.            0.            0.            0.            0.        ]
 [   0.          772.22759756    0.            0.            0.
     0.            0.            0.            0.            0.        ]
 [   0.            0.          828.79163926    0.            0.
     0.            0.            0.            0.            0.        ]
 [   0.            0.            0.          885.47567125    0.
     0.            0.            0.            0.            0.        ]
 [   0.            0.            0.            0.          887.69651531
     0.            0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
   969.23388728    0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.         1213.54561358    0.            0.            0.        ]
 [   0.       

In [33]:
#creating predictions
predicted = np.dot(np.dot(U, sigma), Vt)
predicted_ratings = np.dot(np.dot(U, sigma), Vt) * user_means

In [34]:
predicted_ratings

array([[ 4.13204703e-02,  2.86849575e-02,  6.62762788e-01, ...,
         2.71406035e-02,  6.36375179e-02,  1.12078002e-03],
       [ 4.36798221e-03,  4.92118098e-02,  2.76978840e-01, ...,
         1.80681660e-02, -3.72138894e-03,  1.56169105e-04],
       [-1.35259823e-03,  1.38577574e-04, -6.50964488e-02, ...,
         4.88776468e-04,  6.64276776e-02,  4.31446527e-05],
       ...,
       [-1.54807292e-03,  2.53200421e-02,  2.33365803e-01, ...,
         8.15698056e-03,  3.27823939e-03, -6.74439125e-05],
       [ 2.72674392e-02, -2.54708924e-02,  2.11709355e-01, ...,
         2.87289514e-04, -8.85277957e-03,  3.62645916e-04],
       [ 3.24977410e-02, -3.86551318e-02,  4.96014599e-01, ...,
         3.67927251e-02,  5.64052327e-02,  7.08069591e-04]])

In [35]:
#creating a dataframe for the predictions
predicted_df = pd.DataFrame(predicted_ratings, columns= user_movies.columns)
#Data frame index starts with 0 but original dataset starts with 1, so adding 1 to index
predicted_df.index = predicted_df.index + 1

In [36]:
predicted_df.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
1,0.041320,0.028685,0.662763,1.618436,0.062006,0.596432,0.493691,3.074301,0.029426,0.654572,...,0.000742,0.036697,0.003545,2.302366,0.616471,0.211703,0.527240,0.027141,0.063638,0.001121
2,0.004368,0.049212,0.276979,0.208428,-0.094744,0.298444,-0.035093,0.003464,0.028983,0.070455,...,-0.000120,0.027676,-0.000288,-0.060491,-0.086959,-0.036478,0.243128,0.018068,-0.003721,0.000156
3,-0.001353,0.000139,-0.065096,-0.057096,0.193164,0.113626,-0.000547,0.080283,-0.012290,-0.025152,...,-0.000016,0.059982,0.000064,0.014946,0.031229,0.009437,0.032536,0.000489,0.066428,0.000043
4,-0.003586,0.009182,0.021582,-0.030150,0.239499,-0.012419,0.011642,0.077223,0.002664,0.008873,...,-0.000005,0.033490,0.000052,0.029382,0.037159,0.011544,0.123791,-0.013921,0.024804,-0.000070
5,0.020626,-0.031499,0.493114,0.699624,0.064580,0.105436,0.297689,1.422237,0.022474,0.296479,...,0.000457,0.009697,0.002108,1.231089,0.401294,0.144115,0.026133,0.001027,0.014038,0.000404


In [41]:
#calculating rmse of the predictions done by svd
rmse_df = pd.concat([svd_movies.mean(), predicted_df.mean()], axis=1)
rmse_df.columns = ['Avg_actual_ratings', 'Avg_predicted_ratings']
print(rmse_df.shape)
rmse_df['item_index'] = np.arange(0, rmse_df.shape[0], 1)
rmse_df.head()

(1664, 2)


,Avg_actual_ratings,Avg_predicted_ratings,item_index
title,,,
'Til There Was You (1997),0.065766,0.017465,0
1-900 (1994),0.080677,-0.003618,1
101 Dalmatians (1996),1.427949,0.289674,2
12 Angry Men (1957),2.221963,0.579578,3
187 (1997),0.884747,0.093123,4


In [42]:
RMSE = round((((rmse_df.Avg_actual_ratings - rmse_df.Avg_predicted_ratings) * 2).mean() * 0.5), 5)
print('\nRMSE SVD Model = {} \n'.format(RMSE))


RMSE SVD Model = 0.80312 



In [37]:
#creating function to get recommendations
def svd_recommender(df_predict, user, umr, number_recomm):
    user_predicted_movies = df_predict.loc[user, :].sort_values(ascending = False)
    original_data = umr.loc[user, :].sort_values(ascending = False)
    already_rated = user_movies.loc[user, :].dropna()
    unrated = list(user_movies.loc[1, pd.isnull(user_movies.loc[user, :])].index)
    
    recommendations = df_predict.loc[user][unrated]
    
    recommendations = pd.DataFrame(recommendations.sort_values(ascending = False).index[:number_recomm])
    
    return recommendations, already_rated

In [38]:
#getting values for user no. 650
recommend_650, rated_650 = svd_recommender(predicted_df, 650, svd_movies, 10)

In [39]:
print('Top 10 Recommendations for User 650')
print(recommend_650)

Top 10 Recommendations for User 650
                                title
0             Schindler's List (1993)
1    Shawshank Redemption, The (1994)
2                    Apollo 13 (1995)
3          Usual Suspects, The (1995)
4                  Stand by Me (1986)
5                         Babe (1995)
6                Seven (Se7en) (1995)
7  Four Weddings and a Funeral (1994)
8                        Glory (1989)
9       Much Ado About Nothing (1993)


In [40]:
print('Best rated movies by the user')
rated_650.sort_values(ascending = False)[:10]

Best rated movies by the user


title
Maltese Falcon, The (1941)    5.0
Young Frankenstein (1974)     5.0
Casablanca (1942)             5.0
Citizen Kane (1941)           5.0
Vertigo (1958)                5.0
Star Wars (1977)              5.0
Great Race, The (1965)        5.0
Lawrence of Arabia (1962)     5.0
Princess Bride, The (1987)    5.0
North by Northwest (1959)     5.0
Name: 650, dtype: float64